# Loading The Packages

In [1]:
import pandas as pd
import re

import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import networkx as nx
import pickle
import string
from collections import Counter, defaultdict
import operator

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import gensim
from gensim import corpora


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Loading the Data

In [2]:
product_info = pd.read_csv("C:\\Users\\User\\Desktop\\Textanalytics\\project\\data\\product_info.csv")

In [3]:
df1=pd.read_csv("C:\\Users\\User\\Desktop\\Textanalytics\\project\\data\\reviews_0-250.csv")

C:\Users\User\AppData\Local\Temp\ipykernel_25364\2585637512.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df1=pd.read_csv("C:\\Users\\User\\Desktop\\Textanalytics\\project\\data\\reviews_0-250.csv")


In [4]:
df2=pd.read_csv("C:\\Users\\User\\Desktop\\Textanalytics\\project\\data\\reviews_1250-end.csv")

C:\Users\User\AppData\Local\Temp\ipykernel_25364\573283978.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df2=pd.read_csv("C:\\Users\\User\\Desktop\\Textanalytics\\project\\data\\reviews_1250-end.csv")


In [5]:
df3=pd.read_csv("C:\\Users\\User\\Desktop\\Textanalytics\\project\\data\\reviews_250-500.csv")

In [6]:
df4=pd.read_csv("C:\\Users\\User\\Desktop\\Textanalytics\\project\\data\\reviews_500-750.csv")

In [7]:
df5=pd.read_csv("C:\\Users\\User\\Desktop\\Textanalytics\\project\\data\\reviews_750-1250.csv")

C:\Users\User\AppData\Local\Temp\ipykernel_25364\2464030112.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df5=pd.read_csv("C:\\Users\\User\\Desktop\\Textanalytics\\project\\data\\reviews_750-1250.csv")


In [8]:
df1.shape

(602130, 19)

In [9]:
df2.shape

(49977, 19)

In [10]:
df3.shape

(206725, 19)

In [11]:
df4.shape

(116262, 19)

In [12]:
df5.shape

(119317, 19)

In [13]:
###Concatenating the mutiple dataset for reviews

In [14]:
reviews_df = pd.concat([df1, df2,df3,df4,df5], axis=0, ignore_index=True)

In [15]:
reviews_df.shape

(1094411, 19)

In [16]:
reviews_df.head(3)

,Unnamed: 0,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd
0,0,1741593524,5,1.0,1.0,2,0,2,2023-02-01,I use this with the Nudestix “Citrus Clean Bal...,Taught me how to double cleanse!,NaN,brown,dry,black,P504322,Gentle Hydra-Gel Face Cleanser,NUDESTIX,19.0
1,1,31423088263,1,0.0,NaN,0,0,0,2023-03-21,I bought this lip mask after reading the revie...,Disappointed,NaN,NaN,NaN,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
2,2,5061282401,5,1.0,NaN,0,0,0,2023-03-21,My review title says it all! I get so excited ...,New Favorite Routine,light,brown,dry,blonde,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0


In [17]:
reviews_df = reviews_df.drop('Unnamed: 0', axis=1)

In [18]:
reviews_df.head(3)

,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd
0,1741593524,5,1.0,1.0,2,0,2,2023-02-01,I use this with the Nudestix “Citrus Clean Bal...,Taught me how to double cleanse!,NaN,brown,dry,black,P504322,Gentle Hydra-Gel Face Cleanser,NUDESTIX,19.0
1,31423088263,1,0.0,NaN,0,0,0,2023-03-21,I bought this lip mask after reading the revie...,Disappointed,NaN,NaN,NaN,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
2,5061282401,5,1.0,NaN,0,0,0,2023-03-21,My review title says it all! I get so excited ...,New Favorite Routine,light,brown,dry,blonde,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0


In [19]:
product_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8494 entries, 0 to 8493
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_id          8494 non-null   object 
 1   product_name        8494 non-null   object 
 2   brand_id            8494 non-null   int64  
 3   brand_name          8494 non-null   object 
 4   loves_count         8494 non-null   int64  
 5   rating              8216 non-null   float64
 6   reviews             8216 non-null   float64
 7   size                6863 non-null   object 
 8   variation_type      7050 non-null   object 
 9   variation_value     6896 non-null   object 
 10  variation_desc      1250 non-null   object 
 11  ingredients         7549 non-null   object 
 12  price_usd           8494 non-null   float64
 13  value_price_usd     451 non-null    float64
 14  sale_price_usd      270 non-null    float64
 15  limited_edition     8494 non-null   int64  
 16  new   

In [20]:
reviews_df['submission_time'] = pd.to_datetime(reviews_df['submission_time'])

# Find the oldest and newest date
oldest_date = reviews_df['submission_time'].min()
newest_date = reviews_df['submission_time'].max()

print(f"The oldest date is: {oldest_date}")
print(f"The newest date is: {newest_date}")


The oldest date is: 2008-08-28 00:00:00
The newest date is: 2023-03-21 00:00:00


In [21]:
result_df = pd.merge(reviews_df, product_info[['product_id','primary_category', 'secondary_category', 'tertiary_category']], on='product_id', how='left')

In [30]:
result_df.shape

(1094411, 21)

In [22]:
unique_products = result_df['product_id'].unique()

print(unique_products)
len(unique_products)

['P504322' 'P420652' 'P7880' ... 'P502745' 'P474936' 'P500101']


2351

##### Retaining the reviwes only from 2022-01-01 to 2023 march onwards - since the orginal data is too big and difficult for taining on regular latops 

In [23]:
#reviews_df = result_df[result_df['submission_time'] >= '2022-01-01']

##### only for the year 2022

In [24]:
latest_review = result_df[(result_df['submission_time'] >= '2022-01-01') & (result_df['submission_time'] <= '2022-12-31')]

In [25]:
latest_review.shape

(192227, 21)

In [26]:
reviews_df.head(3)

,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd
0,1741593524,5,1.0,1.0,2,0,2,2023-02-01,I use this with the Nudestix “Citrus Clean Bal...,Taught me how to double cleanse!,NaN,brown,dry,black,P504322,Gentle Hydra-Gel Face Cleanser,NUDESTIX,19.0
1,31423088263,1,0.0,NaN,0,0,0,2023-03-21,I bought this lip mask after reading the revie...,Disappointed,NaN,NaN,NaN,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
2,5061282401,5,1.0,NaN,0,0,0,2023-03-21,My review title says it all! I get so excited ...,New Favorite Routine,light,brown,dry,blonde,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0


In [28]:
category_counts_one

NameError: name 'category_counts_one' is not defined

In [29]:
unique_products_one = latest_review['product_id'].unique()

print(unique_products_one)
len(unique_products_one)

['P420652' 'P7880' 'P218700' ... 'P502745' 'P474936' 'P500101']


2076